In [10]:
import os
import re
from typing import List, TypedDict

from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.tools import tool
from langchain_community.document_loaders import TextLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.messages import (AIMessage, BaseMessage, HumanMessage,
                                     ToolMessage)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.graph import END, StateGraph


load_dotenv()

with open('../data/cafe_menu.txt', 'r', encoding='utf-8') as f:
    file_content = f.read()

menu_items = file_content.strip().split('\n\n')
docs = [Document(page_content=item) for item in menu_items]

embeddings = OpenAIEmbeddings()
db_path = '../../db/cafe_db'
db = FAISS.from_documents(docs, embeddings)
db.save_local(db_path)

tavily_search_func = TavilySearchResults(max_results=1, name="tavily_search")

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 정보를 검색하고 그 결과를 요약합니다. 커피 역사, 음료 제조법 등 일반적인 지식에 사용됩니다."""
    wikipedia = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=2000)
    return wikipedia.run(query)

@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 데이터베이스에서 메뉴의 가격, 재료, 설명 등의 정보를 검색합니다."""
    vector_db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    retriever = vector_db.as_retriever(search_kwargs={"k": 1})
    retrieved_docs = retriever.get_relevant_documents(query)
    return retrieved_docs[0].page_content if retrieved_docs else "관련 메뉴 정보를 찾을 수 없습니다."

tools = [tavily_search_func, wiki_summary, db_search_cafe_func]
llm = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tools)

def run_tool_chain(user_input: str):
    response = llm.invoke([HumanMessage(content=user_input)])
    if not response.tool_calls:
        return response.content
    tool_outputs = []
    for tool_call in response.tool_calls:
        tool_name = tool_call["name"]
        print(f"Tool Calling: '{tool_name}' with args: {tool_call['args']}")
        if tool_name == "tavily_search":
            output = tavily_search_func.invoke(tool_call["args"]["query"])
        elif tool_name == "wiki_summary":
            output = wiki_summary.invoke(tool_call["args"]["query"])
        elif tool_name == "db_search_cafe_func":
            output = db_search_cafe_func.invoke(tool_call["args"]["query"])
        else:
            raise ValueError(f"Unknown tool: {tool_name}")
        tool_outputs.append(ToolMessage(content=str(output), tool_call_id=tool_call["id"]))
    final_response = llm.invoke([HumanMessage(content=user_input), response] + tool_outputs)
    return final_response.content

question = "아메리카노의 가격과 특징은 무엇인가요?"
answer = run_tool_chain(question)
print(f"\n[질문]: {question}")
print(f"[답변]: {answer}")
print("\n문제 4-1 실행 완료\n")


print("문제 4-2: LangGraph 메뉴 추천 시스템 시작")


class CafeState(TypedDict):
    messages: List[BaseMessage]

def extract_menu_info(doc_content: str) -> dict:
    menu_name_match = re.search(r'\d+\.\s*(.+)', doc_content)
    menu_name = menu_name_match.group(1).strip() if menu_name_match else '이름 정보 없음'
    price_match = re.search(r'가격:\s*(.+)', doc_content)
    price = price_match.group(1).strip() if price_match else "가격 정보 없음"
    description_match = re.search(r'설명:\s*(.+)', doc_content, re.DOTALL)
    description = description_match.group(1).strip() if description_match else "설명 없음"
    return {"name": menu_name, "price": price, "description": description}

menu_db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)

def classify_question_node(state: CafeState):
    """사용자의 질문을 키워드 기반으로 먼저 분류하고, 없으면 메뉴 조회를 시도합니다."""
    user_message = state["messages"][-1].content.lower()
    
    
    if "전체" in user_message and "가격" in user_message:
        return "price_inquiry"
    elif "추천" in user_message or "어떤" in user_message or "마실" in user_message:
        return "recommendation_request"
        

    return "menu_inquiry"

def price_info_node(state: CafeState):
    response_text = "카페 메뉴 전체 가격 정보입니다:\n\n"
    for doc in docs:
        info = extract_menu_info(doc.page_content)
        response_text += f"- {info['name']}: {info['price']}\n"
    return {"messages": [AIMessage(content=response_text)]}

def recommend_menu_node(state: CafeState):
    user_message = state["messages"][-1].content
    retrieved_docs = menu_db.similarity_search(user_message, k=3)
    if not retrieved_docs:
        retrieved_docs = menu_db.similarity_search("인기 메뉴", k=3)
    response_text = "이런 메뉴는 어떠신가요? ☕\n\n"
    for doc in retrieved_docs:
        info = extract_menu_info(doc.page_content)
        response_text += f"**{info['name']}**: {info['description']}\n"
    return {"messages": [AIMessage(content=response_text)]}

def menu_info_node(state: CafeState):
    user_message = state["messages"][-1].content
    retrieved_docs = menu_db.similarity_search(user_message, k=1)
    if retrieved_docs:
        info = extract_menu_info(retrieved_docs[0].page_content)
        response_text = f"**{info['name']}**\n- **가격**: {info['price']}\n- **설명**: {info['description']}"
    else:
        response_text = "죄송합니다, 요청하신 메뉴 정보를 찾을 수 없습니다."
    return {"messages": [AIMessage(content=response_text)]}

graph_builder = StateGraph(CafeState)
graph_builder.add_node("price_inquiry", price_info_node)
graph_builder.add_node("recommendation_request", recommend_menu_node)
graph_builder.add_node("menu_inquiry", menu_info_node)
graph_builder.add_conditional_edges(
    "__start__",
    classify_question_node,
    {
        "price_inquiry": "price_inquiry",
        "recommendation_request": "recommendation_request",
        "menu_inquiry": "menu_inquiry",
    }
)
graph_builder.add_edge("price_inquiry", END)
graph_builder.add_edge("recommendation_request", END)
graph_builder.add_edge("menu_inquiry", END)
app = graph_builder.compile()

test_questions = [
    "아메리카노에 대해 알려줘",
    "메뉴 전체 가격이 어떻게 되나요?",
    "달달하고 시원한 음료 추천해 줄래?",
    "카푸치노는 얼마야?"
]

for q in test_questions:
    print(f"\n--- 질문: {q} ---")
    result = app.invoke({"messages": [HumanMessage(content=q)]})
    print(result['messages'][-1].content)
    
print("\n문제 4-2 실행 완료")

Tool Calling: 'db_search_cafe_func' with args: {'query': '아메리카노'}

[질문]: 아메리카노의 가격과 특징은 무엇인가요?
[답변]: 아메리카노의 가격은 ₩4,500입니다. 아메리카노는 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피로, 원두 본연의 맛을 가장 잘 느낄 수 있는 음료입니다. 깔끔하고 깊은 풍미가 특징이며, 설탕이나 시럽을 추가할 수 있습니다.

문제 4-1 실행 완료

문제 4-2: LangGraph 메뉴 추천 시스템 시작

--- 질문: 아메리카노에 대해 알려줘 ---
**아메리카노**
- **가격**: ₩4,500
- **설명**: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.

--- 질문: 메뉴 전체 가격이 어떻게 되나요? ---
카페 메뉴 전체 가격 정보입니다:

- 아메리카노: ₩4,500
- 카페라떼: ₩5,500
- 카푸치노: ₩5,000
- 바닐라 라떼: ₩6,000
- 카라멜 마키아토: ₩6,500
- 콜드브루: ₩5,000
- 프라푸치노: ₩7,000
- 녹차 라떼: ₩5,800
- 아이스 아메리카노: ₩4,500
- 티라미수: ₩7,500


--- 질문: 달달하고 시원한 음료 추천해 줄래? ---
이런 메뉴는 어떠신가요? ☕

**바닐라 라떼**: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.
**녹차 라떼**: 고급 말차 파우더와 부드러운 스팀 밀크로 만든 건강한 음료입니다. 녹차의 은은한 쓴맛과 우유의 부드러움이 조화를 이루며, 항산화 성분이 풍부합니다. 달콤함 조절이 가능합니다.
**프라푸치노**: 에스프레소와 우유, 얼음을 블렌더에 갈아 만든 시원한 음료입니다. 부드럽고 크리미한 질감이 특징이며, 휘핑크림을